# Preprocessing the data

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv("../../data/train.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)
df_valid = pd.read_csv("../../data/validation.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)
df_test = pd.read_csv("../../data/test.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)

df = pd.concat([df_train, df_valid, df_test])

In [6]:
data = train_d+valid_d #[:7000]+valid_d[:2000]
labels = train_l+valid_l #[:7000]+valid_l[:2000]

In [3]:
df.head()

,,,,,quadrant,lyrics
213754,-0.682725,0.316758,Dark Tranquillity,Insanity's Crescendo,1,"Gently, hold our heads Gently, hold our heads ..."
216752,-1.400403,1.538229,Anorexia Nervosa,Stabat mater dolorosa,1,We are the Sun We are the dead stars We are th...
254391,0.815393,0.662457,Uniting Nations,Out Of Touch,0,"You're out of touch, I'm out of time But I'm o..."
266313,1.257460,1.086515,The Walkmen,Brandy alexander,0,Finally close the door You'd left open wide Lo...
283306,0.373325,-0.923151,Duke Ellington,Caravan,3,Night and stars above that shine so bright The...


In [4]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_preprocessing(text):
    text = text.lower() # lower text
    text = replace_contractions(text) # remove contactions
    text = "".join("".join(text).replace("\n", " ").replace("\r", " ")) # remove \n and \r
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace symbols with space
    text = re.sub(r'[0-9]+', '', text)
    text = BAD_SYMBOLS_RE.sub('', text) # replace bad characters with nothing
    text = text.strip()
    text = " ".join([word for word in text.split() if word not in STOPWORDS]) # remove stopwords
    
    return text

In [5]:
df["lyrics"] = df["lyrics"].apply(text_preprocessing)

In [6]:
df.head()

,,,,,quadrant,lyrics
213754,-0.682725,0.316758,Dark Tranquillity,Insanity's Crescendo,1,gently hold heads gently hold heads high aimle...
216752,-1.400403,1.538229,Anorexia Nervosa,Stabat mater dolorosa,1,sun dead stars black sky invading room candle ...
254391,0.815393,0.662457,Uniting Nations,Out Of Touch,0,touch time head around touch time head around ...
266313,1.257460,1.086515,The Walkmen,Brandy alexander,0,finally close door would left open wide locked...
283306,0.373325,-0.923151,Duke Ellington,Caravan,3,night stars shine bright mystery fading light ...


In [7]:
# get lens and remove anything with less than 10
df["len"] = df["lyrics"].apply(len)
df.describe()

,quadrant,len
count,17367.000000,17367.000000
mean,1.346980,1226.929752
std,1.117818,11881.868344
min,0.000000,0.000000
25%,0.000000,453.000000
50%,1.000000,621.000000
75%,2.000000,846.500000
max,3.000000,484895.000000


In [9]:
df = df.loc[df["len"] >= 10]
df.describe()

,quadrant,len
count,17365.000000,17365.000000
mean,1.346905,1227.070659
std,1.117809,11882.545351
min,0.000000,11.000000
25%,0.000000,453.000000
50%,1.000000,621.000000
75%,2.000000,847.000000
max,3.000000,484895.000000


In [10]:
df.drop(labels=["len"], axis=1, inplace=True)
df.head()

,,,,,quadrant,lyrics
213754,-0.682725,0.316758,Dark Tranquillity,Insanity's Crescendo,1,gently hold heads gently hold heads high aimle...
216752,-1.400403,1.538229,Anorexia Nervosa,Stabat mater dolorosa,1,sun dead stars black sky invading room candle ...
254391,0.815393,0.662457,Uniting Nations,Out Of Touch,0,touch time head around touch time head around ...
266313,1.257460,1.086515,The Walkmen,Brandy alexander,0,finally close door would left open wide locked...
283306,0.373325,-0.923151,Duke Ellington,Caravan,3,night stars shine bright mystery fading light ...


In [11]:
train_test = 0.1
train_valid = 0.8

train, test = train_test_split(df, test_size=train_test, random_state=0)
train, valid = train_test_split(train, train_size=train_valid, random_state=0)

In [12]:
train.shape, test.shape, valid.shape

((12502, 2), (1737, 2), (3126, 2))

In [13]:
train.to_csv("preprocessed_train.csv", index=False)
test.to_csv("preprocessed_test.csv", index=False)
valid.to_csv("preprocessed_valid.csv", index=False)

# Data Loading 

In [1]:
import torch
from torchtext.legacy.data import Field, TabularDataset, BucketIterator
from torchtext.vocab import Vectors

In [2]:
EMBED_SIZE = 300
HIDDEN_SIZE = 32
HIDDEN_LAYERS = 2
BIDIRECTIONAL = True
OUTPUT_SIZE = 4
MAX_EPOCHS = 15
LR = 0.5
BATCH_SIZE = 64
MAX_SEQUENCE_LENGTH = 250
DROPOUT_KEEP = 0.8

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
import spacy
NLP = spacy.load("en_core_web_sm")
tokenizer = lambda sent: [x.text for x in NLP.tokenizer(sent) if x.text != " "]

In [5]:
quadrants_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.long, is_target=True)

lyrics_field = Field(tokenize=tokenizer, lower=True, include_lengths=False, batch_first=True, fix_length=MAX_SEQUENCE_LENGTH)
fields = [("quadrant", quadrants_field), ("lyrics", lyrics_field)]

trainset, validset, testset = TabularDataset.splits(path=".", train="preprocessed_train.csv",
                                                   validation="preprocessed_valid.csv",
                                                   test="preprocessed_test.csv", format="CSV",
                                                   fields=fields, skip_header=True)

In [6]:
trainiter = BucketIterator(dataset=trainset, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.lyrics),
                            device=device, sort=True, sort_within_batch=True, train=True)

validiter = BucketIterator(dataset=validset, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.lyrics),
                            device=device, sort=True, sort_within_batch=True, train=True)

testiter = BucketIterator(dataset=testset, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.lyrics),
                            device=device, sort=True, sort_within_batch=True, train=False)

In [7]:
# vocabulary and word embeddings
lyrics_field.build_vocab(trainset, min_freq=3, vectors=Vectors("glove.840B.300d.txt"))
WORD_EMBEDDINGS = lyrics_field.vocab.vectors
VOCAB = lyrics_field.vocab

# Building the Model 

In [8]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import time

In [9]:
VOCAB_SIZE = len(VOCAB)

In [10]:
class BiLSTM(nn.Module):
    
    def __init__(self, vocab_size, word_embeddings):
        super(BiLSTM, self).__init__()
        
        # embedding layer
        self.embeddings = nn.Embedding(vocab_size, EMBED_SIZE)
        self.embeddings.weight = nn.Parameter(WORD_EMBEDDINGS, requires_grad=False)

        # bi-directional LSTM
        self.lstm = nn.LSTM(EMBED_SIZE, HIDDEN_SIZE, bidirectional=BIDIRECTIONAL, 
                            num_layers=HIDDEN_LAYERS, batch_first=True, dropout=DROPOUT_KEEP)
        
        # fully connected layers
        self.fc1 = nn.Linear(HIDDEN_SIZE*HIDDEN_LAYERS*(1+BIDIRECTIONAL), 64)
        self.fc2 = nn.Linear(64, OUTPUT_SIZE)
        
        # ReLU and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(DROPOUT_KEEP)
        


    def forward(self, x):
        embedding_sent = self.embeddings(x)
        lstm_out, (h_n, c_n) = self.lstm(embedding_sent)
        feature_map = self.dropout(h_n)
        feature_map_linear = torch.cat([feature_map[i,:,:] for i in range(feature_map.shape[0])], dim=1)
        hidden = self.relu(self.fc1(feature_map_linear))
        output = self.fc2(hidden)
        
        return output

In [11]:
model = BiLSTM(VOCAB_SIZE, WORD_EMBEDDINGS)
model.to(device)

BiLSTM(
  (embeddings): Embedding(33951, 300)
  (lstm): LSTM(300, 32, num_layers=2, batch_first=True, dropout=0.8, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=4, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.8, inplace=False)
)

# Training the model

In [12]:
def train_func(data, data_):

    # Train the model
    train_loss = 0
    train_acc = 0
    for (lyrics, label) in data:
        optimizer.zero_grad()
        label, lyrics = label.to(device), lyrics.to(device)
        output = model(lyrics)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == label).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(data_), train_acc / len(data_)

def test(data, data_):
    loss = 0
    acc = 0
    for (lyrics, label) in data:
        label, lyrics = label.to(device), lyrics.to(device)
        with torch.no_grad():
            output = model(lyrics)
            loss = criterion(output, label)
            loss += loss.item()
            acc += (output.argmax(1) == label).sum().item()

    return loss / len(data_), acc / len(data_)

In [13]:
N_EPOCHS = MAX_EPOCHS

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, amsgrad=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 7)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(trainiter, trainset)
    valid_loss, valid_acc = test(validiter, validset)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 9 seconds
	Loss: 0.0760(train)	|	Acc: 36.6%(train)
	Loss: 0.0041(valid)	|	Acc: 19.5%(valid)
Epoch: 2  | time in 0 minutes, 9 seconds
	Loss: 0.0391(train)	|	Acc: 37.0%(train)
	Loss: 0.0060(valid)	|	Acc: 18.7%(valid)
Epoch: 3  | time in 0 minutes, 9 seconds
	Loss: 0.0410(train)	|	Acc: 37.0%(train)
	Loss: 0.0010(valid)	|	Acc: 18.7%(valid)
Epoch: 4  | time in 0 minutes, 9 seconds
	Loss: 0.0317(train)	|	Acc: 36.9%(train)
	Loss: 0.0029(valid)	|	Acc: 29.0%(valid)
Epoch: 5  | time in 0 minutes, 9 seconds
	Loss: 0.0335(train)	|	Acc: 37.0%(train)
	Loss: 0.0009(valid)	|	Acc: 29.2%(valid)
Epoch: 6  | time in 0 minutes, 9 seconds
	Loss: 0.0269(train)	|	Acc: 37.0%(train)
	Loss: 0.0019(valid)	|	Acc: 18.7%(valid)
Epoch: 7  | time in 0 minutes, 9 seconds
	Loss: 0.0229(train)	|	Acc: 37.0%(train)
	Loss: 0.0010(valid)	|	Acc: 18.7%(valid)
Epoch: 8  | time in 0 minutes, 9 seconds
	Loss: 0.0247(train)	|	Acc: 38.1%(train)
	Loss: 0.0056(valid)	|	Acc: 32.3%(valid)
Epoch: 9  | time

In [14]:
# save the model
torch.save(model.state_dict(), "model_lstm_glove.bin")

# Evaluating the model

In [15]:
# load the saved model
model.load_state_dict(torch.load("model_lstm_glove.bin", map_location="cpu"))

<All keys matched successfully>

In [16]:
from sklearn.metrics import classification_report

actual_class = []
pred_class = []

for (lyrics, label) in testiter:
        label, lyrics = label.to(device), lyrics.to(device)
        with torch.no_grad():
            output = model(lyrics)
        
        actual_class+=label.cpu().numpy().squeeze().tolist()
        pred_class+=output.argmax(1).cpu().numpy().squeeze().tolist()

In [17]:
print(classification_report(actual_class, pred_class))

              precision    recall  f1-score   support

           0       0.34      1.00      0.50       584
           1       1.00      0.00      0.01       335
           2       0.00      0.00      0.00       487
           3       0.00      0.00      0.00       331

    accuracy                           0.34      1737
   macro avg       0.33      0.25      0.13      1737
weighted avg       0.31      0.34      0.17      1737



/home/martinoywa/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martinoywa/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martinoywa/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
